# Construcción del Dataset Maestro de Valoración Inmobiliaria en Bogotá

Este cuaderno contiene el proceso paso a paso para la construcción del **dataset maestro georreferenciado**, utilizado en el análisis de tendencias del mercado inmobiliario en Bogotá. A partir de diferentes fuentes de datos abiertos, se integran capas de información espacial y contextual sobre las manzanas de la ciudad, como:

- Valor del metro cuadrado
- Estrato socioeconómico
- Localidad y zonificación POT
- Proximidad a estaciones de TransMilenio
- Cercanía a instituciones educativas
- Indicadores de criminalidad

Cada etapa del proceso está documentada con explicaciones sencillas, usando nombres de variables en español y manteniendo las geometrías por capas (`geometry_manzana`, `geometry_estaciones`, etc.), para facilitar la comprensión y reutilización del modelo por otros analistas.

---

### Autores

Maestría en Visual Analytics y Big Data (UNIR)  
[Repositorio del proyecto en GitHub](https://github.com/andres-fuentex/tfm-avm-bogota)

**Sergio Andres Fuentes Gomez** 

**Miguel Alejandro González Cardeñoza**  

**Dirección académica:**  
Mariana Ríos Ortegon, Universidad Internacional de La Rioja (UNIR)

---




## Librerías necesarias para el trabajo

En esta sección se importan las librerías que se utilizarán a lo largo del cuaderno. Si alguna no se encuentra instalada en el entorno, se debe instalar

En este proyecto se utilizarán principalmente:

- `pandas`: para manejo de datos tabulares
- `geopandas`: para manejo de datos espaciales
- `matplotlib.pyplot`: para visualizaciones simples
- `shapely`: para operaciones geométricas


In [71]:
# Librerías de datos y geoprocesamiento
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import re
from pandas.tseries.offsets import MonthEnd

## Paso 1: Cargar los datos de valor del metro cuadrado por manzana

En este paso se cargan los datos geoespaciales con información del valor del metro cuadrado por manzana en Bogotá. Este conjunto de datos contiene múltiples valores según el uso del suelo (residencial, comercial, industrial, etc.) y una geometría tipo polígono para cada manzana urbana.

Este archivo será la **base del dataset maestro**, ya que la manzana es la unidad mínima en la cual se integrarán todos los demás datos.

El archivo se encuentra en el repositorio del proyecto en GitHub con el nombre:  
datos_brutos/valor_m2_manzana.zip

Se cargará completo para su revisión inicial. En los pasos siguientes se seleccionarán las columnas relevantes, se validará la geometría y se preparará para su integración.



In [3]:
# caraga del data set valor_m2_manzana desde git
url_manzanas = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/valor_m2_manzana.zip"
datos_manzanas = gpd.read_file(url_manzanas)
datos_manzanas.head(3)

,Manzana_Id,VrInt_Resi,VrInt_Com_,VrInt_Depo,VrInt_Indu,VrInt_Dota,VrInt_Bode,Vigencia,Area_m2,geometry
0,006107004,0.00,4489412.84,0.0,0.0,0.0,4275881.56,2024-01-01,17488.405077,"POLYGON ((-74.08570 4.61221, -74.08562 4.61214..."
1,004503014,1221846.15,0.00,0.0,0.0,0.0,2900644.13,2024-01-01,12985.146073,"POLYGON ((-74.13185 4.61534, -74.13191 4.61531..."
2,007309002,0.00,0.00,0.0,0.0,0.0,4096827.09,2024-01-01,9017.794131,"POLYGON ((-74.07671 4.65466, -74.07671 4.65465..."


In [4]:
#eliminamos las columnas que no aportan valor 
datos_manzanas = datos_manzanas.drop(columns=["Vigencia","Area_m2"])

In [5]:
# Renombramos la columna geometry para no perderla cuando unamos con otros datos espaciales
datos_manzanas = datos_manzanas.rename(columns={"geometry": "geometry_manzana"})

### Crear columna valor_m2 como valor representativo por manzana

En esta etapa se crea una nueva columna llamada valor_m2, que servirá como el valor de referencia unificado por manzana.

La lógica es la siguiente:

- Si el valor residencial (VrInt_Resi) es diferente de cero, se toma ese valor.
- Si el valor residencial es cero, se calcula el promedio entre todos los valores disponibles: residencial, comercial, depósito, industrial, dotacional y bodega.

Esto permite tener un único valor estimado del metro cuadrado para cada manzana.


In [6]:
# Creamos la columna valor_m2 con la lógica definida
columnas_valor = [
    "VrInt_Resi", "VrInt_Com_", "VrInt_Depo",
    "VrInt_Indu", "VrInt_Dota", "VrInt_Bode"
]

# Calcular la media entre las 6 columnas para cada fila
datos_manzanas["valor_m2"] = datos_manzanas.apply(
    lambda fila: fila["VrInt_Resi"]
    if fila["VrInt_Resi"] != 0
    else fila[columnas_valor].mean(),
    axis=1
)


In [7]:
#eliminamos las columnas que ya no son necesarias

datos_manzanas = datos_manzanas.drop(columns=["VrInt_Resi", "VrInt_Com_", "VrInt_Depo",
                                              "VrInt_Indu", "VrInt_Dota", "VrInt_Bode"])

### Revisión y tratamiento de valores nulos

En este paso se revisan los valores nulos en el conjunto de datos, especialmente en las columnas clave como valor_m2 y geometry_manzana.

Los valores nulos pueden indicar manzanas sin avalúo o con problemas en los datos espaciales.  
Se eliminarán únicamente aquellas filas que tengan:

- valor_m2 nulo (no hay forma de estimar su valor)
- geometry_manzana nula (no se pueden representar en el espacio)

Este filtrado garantiza que el dataset base esté limpio para los siguientes cruces espaciales.


In [8]:
# Verificar cantidad de valores nulos por columna
datos_manzanas.isnull().sum()

Manzana_Id          0
geometry_manzana    0
valor_m2            0
dtype: int64

In [9]:
datos_manzanas.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14755 entries, 0 to 14754
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Manzana_Id        14755 non-null  object  
 1   geometry_manzana  14755 non-null  geometry
 2   valor_m2          14755 non-null  float64 
dtypes: float64(1), geometry(1), object(1)
memory usage: 345.9+ KB


## Paso 2: Cargar y preparar la información de estratificación por manzana

En este paso se trabaja con el shapefile que contiene la información del estrato socioeconómico por manzana.

Cada fila representa una manzana urbana con su correspondiente nivel de estrato (de 0 a 6).

Este dataset servirá para enriquecer el análisis socioeconómico del territorio al integrarse con el dataset de valores del metro cuadrado (valor_m2).


In [11]:
# Cargar dataset de estratificación por manzana
url_estratos = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/estratificacion_manzana.zip"
estratificacion_manzana = gpd.read_file(url_estratos)
estratificacion_manzana.head(3)


,OBJECTID,CODIGO_MAN,ESTRATO,CODIGO_ZON,CODIGO_CRI,NORMATIVA,ACTO_ADMIN,NUMERO_ACT,FECHA_ACTO,ESCALA_CAP,FECHA_CAPT,RESPONSABL,SHAPE_AREA,SHAPE_LEN,geometry
0,695738,00820516,5,15,None,DEC551,DEC,551,2019-09-12,None,2019-09-12,560,21725.000837,676.305821,"POLYGON ((-74.05448 4.64394, -74.05449 4.64392..."
1,695739,00460146,2,5,None,DEC551,DEC,551,2019-09-12,None,2019-09-12,560,1440.019910,183.987736,"POLYGON ((-74.16792 4.64359, -74.16794 4.64361..."
2,695740,00460145,2,4,None,DEC551,DEC,551,2019-09-12,None,2019-09-12,560,6646.540218,506.406054,"POLYGON ((-74.16791 4.64410, -74.16788 4.64412..."


In [12]:
#eliminamos las columnas que no aportan valor 
estratificacion_manzana = estratificacion_manzana.drop(columns=["OBJECTID","CODIGO_CRI","NORMATIVA","ACTO_ADMIN",
                                                               "NUMERO_ACT","FECHA_ACTO","ESCALA_CAP","FECHA_CAPT",
                                                               "RESPONSABL","SHAPE_AREA","SHAPE_LEN"])

In [13]:
# Renombramos la columna geometry para no perderla cuando unamos con otros datos espaciales
estratificacion_manzana = estratificacion_manzana.rename(columns={"geometry": "geometry_estrato"})

### Hallazgo: posible unión por código de manzana

Al revisar la estructura de este dataset, se identifica que la columna `CODIGO_MAN` representa un identificador de manzana de 8 dígitos.

Este código coincide, en la mayoría de los casos, con los primeros 8 caracteres de la columna `Manzana_Id` del dataset anterior (`datos_manzanas`).  
Por lo tanto, es viable realizar una unión entre ambos datasets a través de una transformación que se aplicará más adelante.

Adicionalmente, este dataset incluye una columna `CODIGO_ZON` que representa una agrupación territorial superior.  
Este código será evaluado posteriormente para determinar su utilidad en análisis adicionales o jerarquías espaciales más amplias.


### Revisión de valores nulos

Antes de realizar uniones o transformaciones, es importante verificar si existen valores nulos en el dataset.

En esta etapa se revisan las columnas claves (`CODIGO_MAN`, `ESTRATO`, `geometry_estrato`) para identificar posibles registros incompletos que podrían afectar los análisis o las uniones espaciales posteriores.


In [14]:
# Verificar la cantidad de valores nulos por columna
estratificacion_manzana.isnull().sum()

CODIGO_MAN          0
ESTRATO             0
CODIGO_ZON          0
geometry_estrato    0
dtype: int64

## Paso 3: Cargar y explorar el dataset de delitos de alto impacto

En esta etapa se carga el archivo `delitos_alto_impacto.zip`, el cual contiene información espacial agregada de delitos registrados en Bogotá.

Aún no se tiene certeza sobre la unidad geográfica representada por los polígonos de este shapefile.  
Por tanto, se realizará una exploración inicial para identificar si corresponde a localidades u otra división territorial.

Este dataset será fundamental para incorporar al modelo un indicador de seguridad por área geográfica.


In [15]:
# Cargar dataset de delitos de alto impacto
url_delitos = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/delitos_alto_impacto.zip"
delitos = gpd.read_file(url_delitos)
delitos.head(3)

,CMIULOCAL,CMNOMLOCAL,CMMES,CMH18CONT,CMH19CONT,CMH20CONT,CMHVAR,CMHTOTAL,CMLP18CONT,CMLP19CONT,...,CMHA25CONT,CMHB25CONT,CMHCE25CON,CMHM25CONT,CMHC25CONT,CMDS25CONT,CMVI25CONT,SHAPE_AREA,SHAPE_LEN,geometry
0,09,Fontibón,Ene-Abr (2024vs2025),4.0,7.0,4.0,85.71,378.0,398.0,309.0,...,65.0,168.0,607.0,80.0,102.0,110.0,876.0,0.002710,0.321915,"POLYGON ((-74.15857 4.66274, -74.15848 4.66279..."
1,19,Ciudad Bolívar,Ene-Abr (2024vs2025),83.0,57.0,61.0,61.82,378.0,808.0,781.0,...,61.0,35.0,358.0,115.0,39.0,286.0,1396.0,0.010585,0.702136,"POLYGON ((-74.21092 4.38691, -74.21114 4.38750..."
2,02,Chapinero,Ene-Abr (2024vs2025),5.0,3.0,1.0,-50.00,378.0,323.0,202.0,...,13.0,68.0,951.0,13.0,139.0,77.0,266.0,0.003095,0.333966,"POLYGON ((-74.01116 4.66459, -74.01154 4.66461..."


### Exploración inicial del dataset de delitos de alto impacto

Al revisar la estructura del dataset `delitos_alto_impacto`, se evidencia que los datos están organizados por localidad.

- La columna `CMIULOCAL` representa el código numérico de la localidad.
- La columna `CMNOMLOCAL` contiene el nombre de la localidad (ej. Fontibón, Chapinero).
- Existen múltiples columnas que indican la cantidad de delitos cometidos, desagregados por edad, género y tipo, en una ventana temporal especificada en `CMMES` (ej. Ene-Abr 2024 vs 2025).
- La columna `geometry` contiene polígonos que representan espacialmente el área geográfica de cada localidad.

Este dataset será clave para incorporar un componente de análisis de seguridad al modelo, mediante la agregación de datos delictivos por localidad.


In [16]:
#eliminamos las columnas que no aportan valor 
delitos = delitos.drop(columns=["CMMES","SHAPE_AREA","SHAPE_LEN"])

In [17]:
# Renombramos la columna geometry para no perderla cuando unamos con otros datos espaciales
delitos = delitos.rename(columns={"geometry": "geometry_delitos"})

### Creación de la columna `cantidad_delitos`

Con el fin de tener un indicador agregado de criminalidad por localidad, se crea una nueva columna llamada `cantidad_delitos`.

Esta variable corresponde a la suma de todos los conteos de delitos reportados en las diferentes categorías (por edad, género y tipo) dentro de cada localidad.

Esta columna se utilizará posteriormente como referencia para el análisis espacial de seguridad.

In [18]:
# Crear columna 'cantidad_delitos' sumando todas las columnas numéricas excepto las de identificación y geometría
columnas_delitos = delitos.columns.difference(['CMIULOCAL', 'CMNOMLOCAL', 'geometry_delitos'])
delitos['cantidad_delitos'] = delitos[columnas_delitos].sum(axis=1)

### Cálculo del total de delitos en la ciudad

Se calcula la suma total de delitos reportados en todas las localidades de Bogotá, usando la columna `cantidad_delitos`.

Este valor servirá como referencia general para análisis comparativos y visualizaciones proporcionales por localidad.

In [20]:
delitos["cantidad_delitos"].describe()

count        21.000000
mean     120400.279048
std       21036.051413
min       86421.000000
25%      106991.320000
50%      119202.570000
75%      128766.940000
max      165249.280000
Name: cantidad_delitos, dtype: float64

In [19]:
# Calcular el total de delitos en toda la ciudad
total_delitos = delitos['cantidad_delitos'].sum()
total_delitos

2528405.8600000003

### Clasificación de localidades según niveles de riesgo delictivo

Con base en la distribución estadística de la columna `cantidad_delitos`, se crea una nueva variable categórica llamada `nivel_riesgo_delictivo`.

Esta columna clasifica cada localidad en cuatro niveles según el número total de delitos reportados, utilizando los cuartiles como criterio:

- **Bajo riesgo**: ≤ 25% de delitos
- **Riesgo moderado**: entre 25% y 50%
- **Riesgo medio-alto**: entre 50% y 75%
- **Alto riesgo**: > 75%

Esta clasificación permite representar gráficamente el riesgo delictivo sin emitir juicios ni estigmatizar, conservando un enfoque técnico.


In [21]:
# Clasificación basada en cuartiles
def clasificar_riesgo(valor):
    if valor <= 106991:
        return "Bajo riesgo"
    elif valor <= 119202:
        return "Riesgo moderado"
    elif valor <= 128766:
        return "Riesgo medio-alto"
    else:
        return "Alto riesgo"

delitos['nivel_riesgo_delictivo'] = delitos['cantidad_delitos'].apply(clasificar_riesgo)


### Limpieza final del dataset de delitos

Luego de crear las columnas agregadas `cantidad_delitos` y `nivel_riesgo_delictivo`, se eliminan las columnas de conteo individual de delitos.

Estas columnas fueron utilizadas exclusivamente para el cálculo y no son necesarias para los análisis posteriores.

Se conserva la geometría de la localidad, el nombre, el código, la cantidad total de delitos y la clasificación del riesgo.


In [22]:
# Conservar solo las columnas esenciales
columnas_a_conservar = ['CMIULOCAL', 'CMNOMLOCAL', 'cantidad_delitos', 'nivel_riesgo_delictivo', 'geometry_delitos']
delitos = delitos[columnas_a_conservar]

In [23]:
# Verificar la cantidad de valores nulos por columna
delitos.isnull().sum()

CMIULOCAL                 0
CMNOMLOCAL                0
cantidad_delitos          0
nivel_riesgo_delictivo    0
geometry_delitos          1
dtype: int64

### Eliminación de registros con geometría nula

Durante la revisión de datos nulos, se identificó un registro sin geometría asociada (`geometry_delitos`).

Este registro se elimina para evitar errores en uniones espaciales o visualizaciones.

La limpieza garantiza que todos los registros tengan una representación geográfica válida.


In [24]:
# Eliminar filas sin geometría
delitos = delitos[delitos['geometry_delitos'].notnull()]

## Paso 4: Carga y exploración del dataset de estaciones de TransMilenio

En este paso se carga el archivo `tm_estaciones.geojson`, el cual contiene la ubicación de las estaciones del sistema de transporte público TransMilenio en Bogotá.

Cada estación está representada por un punto geográfico y se encuentra asociada a una troncal del sistema.

Este dataset será útil para analizar la cercanía de las manzanas y localidades a las estaciones de transporte masivo.

In [26]:
# Cargar el dataset de estaciones de TransMilenio
url_transporte = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/tm_estaciones.geojson"
transporte = gpd.read_file(url_transporte)
transporte.head(3)

,fid,nombre_estacion,troncal_estacion,coord_x,coord_y,geometry
0,41,La Campiña,C,-74.090329,4.741825,POINT (-74.09033 4.74183)
1,36,Pepe Sierra,B,-74.055193,4.698793,POINT (-74.05519 4.69879)
2,27,Avenida 68,D,-74.080215,4.685354,POINT (-74.08021 4.68535)


### Reconstrucción de la geometría de estaciones de TransMilenio

Aunque el dataset contiene una columna `geometry`, se observa que las coordenadas `coord_x` y `coord_y` tienen una mayor precisión decimal.

Por lo tanto, se reconstruye la geometría utilizando estas coordenadas para asegurar mayor exactitud en los análisis espaciales.


In [27]:
from shapely.geometry import Point

# Reconstruir la geometría con mayor precisión
transporte['geometry_transporte'] = transporte.apply(lambda row: Point(row['coord_x'], row['coord_y']), axis=1)

In [28]:
# Eliminar columnas innecesarias
transporte = transporte.drop(columns=['fid', 'geometry'])

In [29]:
# Verificar la cantidad de valores nulos por columna
transporte.isnull().sum()

nombre_estacion        0
troncal_estacion       0
coord_x                0
coord_y                0
geometry_transporte    0
dtype: int64

## Paso 5: Carga y exploración del dataset de colegios

Este dataset contiene información detallada sobre los establecimientos educativos de Bogotá, incluyendo:

- Nombre del colegio y sede
- UPZ y UPL donde se ubican
- Coordenadas de ubicación (`COORDENADA LONGITUD (X)` y `COORDENADA LATITUD (Y)`)

Estos datos serán útiles para analizar la disponibilidad de centros educativos por zona. Más adelante se generará una columna geométrica para poder realizar uniones espaciales con otros datos.

es un excel y tiene los datos despues de la fila 11

In [116]:
# Cargar el dataset de estaciones de Colegios Bogota
url_colegios = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/colegios_bogota.xlsx"
colegios = pd.read_excel(url_colegios,header=11)
colegios.head(3)

,No.,#_LOC,NOMBRE_LOCALIDAD,DANE11_ESTABLECIMIENTO_EDUCATIVO,DANE12_ESTABLECIMIENTO_EDUCATIVO,NOMBRE_ESTABLECIMIENTO_EDUCATIVO,NIT (DISTRITALES NIT FSE),DANE11_SEDE_EDUCATIVA,DANE12_SEDE_EDUCATIVA,NOMBRE_SEDE_EDUCATIVA,...,CHIP_1,CPF,ESTRATO_GEO,No_UPZ,NOMBRE_UPZ,SECTOR_CENSAL,COORDENADA LONGITUD (X),COORDENADA LATITUD (Y),NUMERO_UPL,NOMBRE UPL
0,1,1,USAQUEN,1.110010e+10,111001000132,COLEGIO AQUILEO PARRA (IED),8001053451,11100100136,111001000132,AQUILEO PARRA,...,AAA0142LBRJ,115.0,SIN ESTRATO,9,VERBENAL,008527,-74.039171,4.765726,26.0,TOBERIN
1,2,1,USAQUEN,1.110013e+10,111001029955,COLEGIO AGUSTIN FERNANDEZ (IED),830092890,11100129959,111001029955,AGUSTIN FERNANDEZ,...,AAA0115LEWF,114.0,SIN ESTRATO,13,LOS CEDROS,008520,-74.024102,4.729647,26.0,TOBERIN
2,3,1,USAQUEN,1.110013e+10,111001029955,COLEGIO AGUSTIN FERNANDEZ (IED),830092890,11184800243,111848000244,SAN BERNARDO,...,AAA0115HUFT,128.0,1,11,SAN CRISTOBAL NORTE,008520,-74.017673,4.732533,26.0,TOBERIN


In [117]:
#eliminamos las columnas que no aportan valor 
cols = [
    '#_LOC', 'NOMBRE_LOCALIDAD', 'NOMBRE_ESTABLECIMIENTO_EDUCATIVO',
    'SECTOR', 'BARRIO-GEO', 'ESTRATO_GEO', 'NOMBRE_UPZ',
    'COORDENADA LONGITUD (X)', 'COORDENADA LATITUD (Y)']
colegios = colegios[cols]

In [118]:
colegios.head(3)

,#_LOC,NOMBRE_LOCALIDAD,NOMBRE_ESTABLECIMIENTO_EDUCATIVO,SECTOR,BARRIO-GEO,ESTRATO_GEO,NOMBRE_UPZ,COORDENADA LONGITUD (X),COORDENADA LATITUD (Y)
0,1,USAQUEN,COLEGIO AQUILEO PARRA (IED),OFICIAL,EL VERBENAL 1,SIN ESTRATO,VERBENAL,-74.039171,4.765726
1,1,USAQUEN,COLEGIO AGUSTIN FERNANDEZ (IED),OFICIAL,BOSQUE DE PINOS III,SIN ESTRATO,LOS CEDROS,-74.024102,4.729647
2,1,USAQUEN,COLEGIO AGUSTIN FERNANDEZ (IED),OFICIAL,CERROS NORTE,1,SAN CRISTOBAL NORTE,-74.017673,4.732533


In [119]:
# Función para extraer el primer número decimal de un string
def extraer_primer_numero(texto):
    if pd.isna(texto):
        return None
    texto = texto.replace(',', '.')
    m = re.search(r'(-?\d+\.\d+)', texto)
    return float(m.group(1)) if m else None

# 3. Limpiar y convertir en float
colegios['lon'] = colegios['COORDENADA LONGITUD (X)'].astype(str).apply(extraer_primer_numero)
colegios['lat'] = colegios['COORDENADA LATITUD (Y)'].astype(str).apply(extraer_primer_numero)


In [120]:
# Crear columna de geometría usando las coordenadas
colegios["geometry_colegios"] = colegios.apply(
    lambda row: Point(row["lon"], row["lat"]),
    axis=1
)

In [121]:
# Convertir en GeoDataFrame y definir el CRS geográfico
colegios = gpd.GeoDataFrame(colegios, geometry="geometry_colegios", crs="EPSG:4326")

### Tratamiento de datos de colegios

En este paso se trabajó con el dataset de colegios de Bogotá, que contiene información sobre la ubicación y características de los establecimientos educativos.

Se seleccionaron las columnas relevantes para el análisis, entre ellas el nombre de la localidad, el establecimiento, el sector (si es oficial o privado), el barrio, el estrato geográfico y las coordenadas de ubicación.

Dado que las coordenadas de longitud y latitud venían en formato de texto y en algunos casos contenían errores o caracteres no numéricos, fue necesario aplicar una función de limpieza para extraer correctamente los valores decimales. Posteriormente, se usaron estas coordenadas para crear una columna de geometría (`geometry_colegios`) que representa la ubicación geográfica de cada colegio.

Finalmente, se convirtió el dataset en un GeoDataFrame y se estableció el sistema de referencia espacial EPSG:4326, que es el estándar geográfico global.


In [122]:
# Verificar la cantidad de valores nulos por columna
colegios.isnull().sum()

#_LOC                                0
NOMBRE_LOCALIDAD                     0
NOMBRE_ESTABLECIMIENTO_EDUCATIVO     0
SECTOR                               0
BARRIO-GEO                           8
ESTRATO_GEO                         49
NOMBRE_UPZ                           4
COORDENADA LONGITUD (X)              0
COORDENADA LATITUD (Y)               0
lon                                  0
lat                                  0
geometry_colegios                    0
dtype: int64

In [123]:
# Eliminamos las columnas que tienen valores nulos para conservar todos los registros de colegios
colegios = colegios.drop(columns=["BARRIO-GEO", "ESTRATO_GEO", "NOMBRE_UPZ","COORDENADA LONGITUD (X)",
                                 "COORDENADA LATITUD (Y)", "lon","lat"])

In [129]:
colegios = colegios.drop(columns=["geometry"])

## Paso 6 Carga del dataset de Áreas de Actividad (POT)

En este paso se carga el dataset `area_actividad_pot.zip`, el cual contiene los polígonos definidos por el Plan de Ordenamiento Territorial (POT) para Bogotá. Este dataset representa diferentes zonas de uso del suelo y su geometría asociada será clave para relacionarla más adelante con las manzanas y otras capas geográficas.

Se visualizan las primeras filas para comprender su estructura y variables disponibles.


In [57]:
# Cargar el dataset de áreas de actividad del POT
url_pot = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/area_actividad_pot.zip"
area_actividad_pot = gpd.read_file(url_pot)
area_actividad_pot.head(3)

,CODIGO_ID,CODIGO_ARE,NOMBRE_ARE,ACTO_ADMIN,NUMERO_ACT,FECHA_ACTO,NORMATIVA,OBSERVACIO,ESCALA_CAP,FECHA_CAPT,RESPONSABL,RULEID,Shape_Leng,Shape_Area,geometry
0,862,AAERAE,Área de Actividad Estructurante - AAE - Recept...,DEC,555,2021-12-29,En el marco del Decreto 555 del 29 de diciembr...,Por el cual se adopta la revisión general del ...,2000,2021-12-29,553,2,2901.008654,1.655190e+05,"POLYGON ((91816.859 95218.099, 91816.875 95217..."
1,863,AAERAE,Área de Actividad Estructurante - AAE - Recept...,DEC,555,2021-12-29,En el marco del Decreto 555 del 29 de diciembr...,Por el cual se adopta la revisión general del ...,2000,2021-12-29,553,2,1998.009835,8.032941e+04,"POLYGON ((92128.884 94748.509, 92088.878 94703..."
2,864,AAERAE,Área de Actividad Estructurante - AAE - Recept...,DEC,555,2021-12-29,En el marco del Decreto 555 del 29 de diciembr...,Por el cual se adopta la revisión general del ...,2000,2021-12-29,553,2,19252.709778,2.055660e+06,"POLYGON ((92829.254 94217.060, 92829.292 94205..."


In [58]:
#eliminamos las columnas que no aportan valor 
area_actividad_pot = area_actividad_pot.drop(columns=["CODIGO_ID","CODIGO_ARE","ACTO_ADMIN","NUMERO_ACT","FECHA_ACTO",
                                                     "NORMATIVA", "OBSERVACIO","ESCALA_CAP","FECHA_CAPT","RESPONSABL",
                                                     "RULEID","Shape_Leng","Shape_Area"])

In [59]:
area_actividad_pot.head(3)

,NOMBRE_ARE,geometry
0,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((91816.859 95218.099, 91816.875 95217..."
1,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((92128.884 94748.509, 92088.878 94703..."
2,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((92829.254 94217.060, 92829.292 94205..."


In [60]:
area_actividad_pot['NOMBRE_ARE'].unique()

array(['Área de Actividad Estructurante - AAE - Receptora de actividades económicas',
       'Área de Actividad Grandes Servicios Metropolitanos - AAGSM',
       'Área de Actividad Estructurante - AAE - Receptora de vivienda de interés social',
       'Área de Actividad de Proximidad - AAP- Generadora de soportes urbanos',
       'Área de Actividad de Proximidad - AAP - Receptora de soportes urbanos',
       'Plan Especial de Manejo y Protección -PEMP BIC Nacional: se rige por lo establecido en la Resolución que lo aprueba o la norma que la modifique o sustituya'],
      dtype=object)

In [61]:
# Diccionario de mapeo para uso interpretativo
mapa_uso_pot = {
    'Área de Actividad Estructurante - AAE - Receptora de actividades económicas': 'Zona Económica',
    'Área de Actividad Grandes Servicios Metropolitanos - AAGSM': 'Grandes Servicios',
    'Área de Actividad Estructurante - AAE - Receptora de vivienda de interés social': 'Vivienda VIS',
    'Área de Actividad de Proximidad - AAP- Generadora de soportes urbanos': 'Servicios Urbanos',
    'Área de Actividad de Proximidad - AAP - Receptora de soportes urbanos': 'Zona Residencial',
    'Plan Especial de Manejo y Protección -PEMP BIC Nacional: se rige por lo establecido en la Resolución que lo aprueba o la norma que la modifique o sustituya': 'Zona Protegida'
}

# Crear la nueva columna
area_actividad_pot["uso_pot_simplificado"] = area_actividad_pot["NOMBRE_ARE"].map(mapa_uso_pot)

In [62]:
area_actividad_pot.head()

,NOMBRE_ARE,geometry,uso_pot_simplificado
0,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((91816.859 95218.099, 91816.875 95217...",Zona Económica
1,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((92128.884 94748.509, 92088.878 94703...",Zona Económica
2,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((92829.254 94217.060, 92829.292 94205...",Zona Económica
3,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((98441.160 94805.668, 98440.488 94805...",Zona Económica
4,Área de Actividad Estructurante - AAE - Recept...,"POLYGON ((90835.575 94198.543, 90843.967 94191...",Zona Económica


### Clasificación interpretativa del uso del suelo POT

Se crea una nueva columna llamada `etiqueta_uso` para describir el tipo de uso del suelo de manera clara y breve, útil para etiquetas visuales:

- **Zona Económica**: áreas para empresas, oficinas y comercio.
- **Grandes Servicios**: zonas con hospitales, universidades o centros públicos.
- **Vivienda VIS**: zonas prioritarias para vivienda de interés social.
- **Servicios Urbanos**: parques, plazas o escuelas al servicio de la comunidad.
- **Zona Residencial**: áreas residenciales con servicios básicos ligados al barrio.
- **Zona Protegida**: áreas reguladas por normativas patrimoniales o ambientales.

Estas etiquetas se basan en la definición oficial de áreas del uso del suelo contenida en el **Decreto Distrital 555 de 2021** del Plan de Ordenamiento Territorial (POT) de Bogotá


In [63]:
# Verificar la cantidad de valores nulos por columna
area_actividad_pot.isnull().sum()

NOMBRE_ARE              0
geometry                0
uso_pot_simplificado    0
dtype: int64

In [99]:
# Renombramos la columna geometry para no perderla cuando unamos con otros datos espaciales
area_actividad_pot = area_actividad_pot.rename(columns={"geometry": "geometry_pot"})

## Paso 7: Carga del Índice de Precios de Vivienda Usada (IPVU)

En este paso incorporamos el Índice de Precios de Vivienda Usada (IPVU), publicado por el Banco de la República, el cual constituye una serie trimestral que refleja la evolución del valor de la vivienda usada en Colombia en términos reales y nominales desde 1988. Este índice permite ajustar los precios históricos al nivel actual de precios y proyectar escenarios futuros en el modelo de valoración automatizada.

El IPVU se construye con base en la metodología de ventas repetidas (Case & Shiller, 1989) y sirve como proxy del comportamiento del mercado inmobiliario para incluir variables macroeconómicas en el análisis de precios por metro cuadrado.

Este conjunto de datos contiene columnas con el valor del índice en diferentes trimestres, tanto en términos nominales como reales. Más adelante se vinculará con el dataset maestro según el periodo de corte del análisis.


In [87]:
# Cargar el dataset del Índice de Precios de Vivienda Usada (IPVU)
url_ipvu = "https://github.com/andres-fuentex/tfm-avm-bogota/raw/main/datos_brutos/ipvu_banco_republica.xlsx"
ipvu = pd.read_excel(url_ipvu, sheet_name="Datos")
ipvu.head(3)

,Fecha,"Índice de precios de la vivienda usada (IPVU), Índice nominal - trimestral","Índice de precios de la vivienda usada (IPVU), Índice real - trimestral"
0,yyyy/mm/dd,Índice,Índice
1,1988/03/31,55.57,100.33
2,1988/06/30,54.23,88.95


In [88]:
# Eliminamos la primera fila
ipvu = ipvu.iloc[1:].reset_index(drop=True)
ipvu.head(3)

,Fecha,"Índice de precios de la vivienda usada (IPVU), Índice nominal - trimestral","Índice de precios de la vivienda usada (IPVU), Índice real - trimestral"
0,1988/03/31,55.57,100.33
1,1988/06/30,54.23,88.95
2,1988/09/30,64.24,99.58


In [89]:
# Filtrar filas no nulas y con formato de fecha válido
ipvu = ipvu[ipvu['Fecha'].notna() & ipvu['Fecha'].str.match(r'^\d{4}/\d{2}/\d{2}$')].copy()
# Renombrar columnas
ipvu.columns = ['fecha', 'ipvu_nominal', 'ipvu_real']
# Convertir la columna 'fecha' a datetime
ipvu['fecha'] = pd.to_datetime(ipvu['fecha'])

In [90]:
ipvu.head(3)

,fecha,ipvu_nominal,ipvu_real
0,1988-03-31,55.57,100.33
1,1988-06-30,54.23,88.95
2,1988-09-30,64.24,99.58


### Incorporación del Índice de Precios de Vivienda Usada (IPVU)

Teniendo en cuenta la utilidad del Índice de Precios de Vivienda Usada (IPVU) publicado por el Banco de la República, y su respaldo metodológico basado en ventas repetidas ajustadas por inflación (Case & Shiller, 1989; Calhoun, 1996), se plantea el siguiente uso dentro del modelo de valoración:

- El IPVU real se usará como un factor de ajuste macroeconómico para actualizar los valores del metro cuadrado (valor_m2) en cada manzana, con el fin de llevarlos a un nivel de precios comparable y homogéneo.

- Para simplificar el análisis y facilitar su integración, se agruparán los datos trimestrales en periodos **semestrales**, y se calculará un **factor de ajuste** para cada semestre respecto a un periodo base de referencia (ejemplo: segundo semestre de 2024).

- Además, se proyectará el IPVU para los **dos próximos años** (hasta 2026), con el fin de incorporar escenarios de valoración futura en el modelo AVM.

Esta integración permitirá capturar el efecto de las condiciones macroeconómicas y del ciclo inmobiliario sobre los precios observados, fortaleciendo la capacidad predictiva y explicativa del modelo.


In [93]:
# 1. Asegurarnos que 'fecha' es tipo datetime
ipvu['fecha'] = pd.to_datetime(ipvu['fecha'])

# 2. Crear columna 'semestre' en formato "YYYY-S1" o "YYYY-S2"
def obtener_semestre(fecha):
    semestre = "S1" if fecha.month <= 6 else "S2"
    return f"{fecha.year}-{semestre}"

ipvu['semestre'] = ipvu['fecha'].apply(obtener_semestre)

# 3. Agrupar por semestre y calcular promedio del índice real
ipvu_semestral = ipvu.groupby('semestre').agg({'ipvu_real': 'mean'}).reset_index()

# 4. Determinar el último semestre real disponible
semestres_disponibles = ipvu_semestral['semestre'].sort_values()
semestre_base = semestres_disponibles.iloc[-1]  # Último semestre real
valor_base_ajuste = ipvu_semestral[ipvu_semestral['semestre'] == semestre_base]['ipvu_real'].values[0]

# 5. Crear proyecciones para 2026 y 2027 (4 semestres)
proyecciones = []
anio_base = int(semestre_base.split("-")[0])
sem_base = semestre_base.split("-")[1]

anio = anio_base
sem = sem_base
ultimo_valor = valor_base_ajuste

for i in range(1, 5):  # 4 semestres futuros
    if sem == "S1":
        nuevo_semestre = f"{anio}-S2"
        sem = "S2"
    else:
        anio += 1
        nuevo_semestre = f"{anio}-S1"
        sem = "S1"
    nuevo_valor = ultimo_valor * (1 + 0.025)**i  # incremento compuesto del 2.5% por semestre
    proyecciones.append({'semestre': nuevo_semestre, 'ipvu_real': nuevo_valor})

# 6. Añadir proyecciones al DataFrame original
proyecciones_df = pd.DataFrame(proyecciones)
ipvu_semestral = pd.concat([ipvu_semestral, proyecciones_df], ignore_index=True)

# 7. Recalcular factor de ajuste tomando como base el valor del semestre_base
valor_base_ajuste = ipvu_semestral[ipvu_semestral['semestre'] == semestre_base]['ipvu_real'].values[0]
ipvu_semestral['factor_ajuste'] = ipvu_semestral['ipvu_real'] / valor_base_ajuste

# 8. Verificación final
ipvu_semestral.tail(6)


,semestre,ipvu_real,factor_ajuste
72,2024-S1,126.5,0.981686
73,2024-S2,128.86,1.0
74,2025-S1,132.0815,1.025
75,2025-S2,135.383538,1.050625
76,2026-S1,138.768126,1.076891
77,2026-S2,142.237329,1.103813


In [94]:
ipvu = ipvu_semestral[ipvu_semestral['semestre'].str.contains('2025|2026|2027')].copy()


## Subconjunto IPVU para el modelo AVM (2025–2027)

Tras la construcción de la serie semestral del Índice de Precios de Vivienda Usada (IPVU), se seleccionaron las observaciones correspondientes a los años 2025, 2026 y 2027. Este subconjunto contiene valores reales del índice (para 2025) y proyecciones estimadas con una tasa de crecimiento del 2.5% semestral (para 2026 y 2027), siguiendo una lógica conservadora basada en la evolución reciente del mercado.

A cada semestre se le asigna un `factor_ajuste`, que permite **actualizar el valor histórico del metro cuadrado** al valor esperado en ese periodo. Este factor parte de un valor base 1.0 para el segundo semestre de 2025 (2025-S2) y se ajusta proporcionalmente en los periodos proyectados.

Este conjunto será clave para aplicar **ajustes macroeconómicos** al modelo hedonista de valoración, reflejando las dinámicas del mercado inmobiliario a corto y mediano plazo.


## Exportación de datasets limpios para integración y visualización

Una vez finalizado el tratamiento individual de cada conjunto de datos (limpieza de columnas irrelevantes, transformación de variables, creación de campos calculados, estandarización de geometrías y análisis de valores nulos), se procede a exportar los datasets resultantes. Estos archivos representan las versiones listas para integración en el dataset maestro y para su posterior uso en visualización geográfica, modelado AVM y análisis exploratorio en herramientas como Power BI, Tableau o plataformas web.

Cada archivo se exporta con formato apropiado (GeoJSON o CSV) y nombres identificables según su origen y propósito. La organización de estos archivos en un repositorio central (por ejemplo, GitHub) facilita su trazabilidad y reutilización en el flujo de trabajo del proyecto.


In [108]:
# 1. Establecer la columna de geometría activa (si no está)
datos_manzanas = datos_manzanas.set_geometry("geometry_manzana")

# 2. Asignar CRS si no tiene
if datos_manzanas.crs is None:
    datos_manzanas = datos_manzanas.set_crs(epsg=3116)  # MAGNA-SIRGAS Bogotá

# 3. Convertir a EPSG:4326 (lat/lon)
datos_manzanas = datos_manzanas.to_crs(epsg=4326)

# 4. Exportar a GeoJSON en el directorio de trabajo de Kaggle
datos_manzanas.to_file("/kaggle/working/datos_manzanas.geojson", driver="GeoJSON")


In [109]:
# 1. Establecer columna de geometría activa
estratificacion_manzana = estratificacion_manzana.set_geometry("geometry_estrato")

# 2. Asignar CRS si no tiene
if estratificacion_manzana.crs is None:
    estratificacion_manzana = estratificacion_manzana.set_crs(epsg=3116)

# 3. Convertir a lat/lon (EPSG:4326)
estratificacion_manzana = estratificacion_manzana.to_crs(epsg=4326)

# 4. Exportar como GeoJSON
estratificacion_manzana.to_file("/kaggle/working/estratificacion_manzana.geojson", driver="GeoJSON")


In [110]:
# 1. Establecer columna de geometría activa
delitos = delitos.set_geometry("geometry_delitos")

# 2. Asignar CRS si no tiene
if delitos.crs is None:
    delitos = delitos.set_crs(epsg=3116)  # MAGNA-SIRGAS Bogotá

# 3. Convertir a lat/lon (EPSG:4326)
delitos = delitos.to_crs(epsg=4326)

# 4. Exportar como GeoJSON
delitos.to_file("/kaggle/working/delitos.geojson", driver="GeoJSON")


In [112]:
# 2. Establecer como geometría activa
transporte = transporte.set_geometry("geometry_transporte")

# 3. Asignar CRS si no lo tiene
if transporte.crs is None:
    transporte = transporte.set_crs(epsg=4326)  # Ya está en lat/lon

# 4. Exportar como GeoJSON
transporte.to_file("/kaggle/working/transporte.geojson", driver="GeoJSON")


In [134]:
import geopandas as gpd

# Asegurarnos de que 'geometry_colegios' sea la única columna geométrica
colegios = colegios.drop(columns=[col for col in colegios.columns if col.startswith('geometry') and col != 'geometry_colegios'])

# Convertir en GeoDataFrame con la columna correcta
colegios = gpd.GeoDataFrame(colegios, geometry="geometry_colegios")

# Asignar CRS si no está definido
colegios = colegios.set_crs(epsg=4326)

# Exportar a GeoJSON
colegios.to_file("/kaggle/working/colegios.geojson", driver="GeoJSON")



In [144]:
# Reproyectar correctamente al sistema WGS84
area_actividad_pot = area_actividad_pot.set_geometry("geometry_pot")
area_actividad_pot = area_actividad_pot.to_crs(epsg=4326)

# Exportar como GeoJSON
area_actividad_pot.to_file("/kaggle/working/area_actividad_pot.geojson", driver="GeoJSON")


In [145]:
# Exportar el dataset semestral de IPVU a CSV
ipvu.to_csv("/kaggle/working/ipvu.csv", index=False)
